<a href="https://colab.research.google.com/github/bradshimmin/llm_experiments/blob/main/AI_Skills_Structured_Outputs_TikTok_13sep24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up software
#
!pip -q install openai tiktoken
print("All done")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00
All done


In [ ]:
# Import libraries
#
import os
import openai
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import textwrap
import tiktoken

from enum import Enum
from typing import Union
from pydantic import BaseModel
from openai import OpenAI

print("all done")

all done


In [ ]:
# Load openai key
#
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI_key')

---
# Set up AI Skills data

In [ ]:
# Mount Google Drive
#
from google.colab import drive
drive.mount('/content/gdrive')

print("all done")

Mounted at /content/gdrive
all done


In [ ]:
# Load our sample dataset using 2q24 cleaned df
#
df_sample = pd.read_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/ai_skills_tracker/ai_skills_2q24/ai_skills_2q24_final_clean_28aug24.pkl")


In [ ]:
df_sample.shape

(205517, 31)

In [ ]:
df_sample.head(n=2)

,RecordID,Job title,Full job description,Date collected,Date posted,Week,Year,Month,Quarter,Hiring entity,...,Salary start,Salary end,Main industry,Job title category,AI technologies,AI skills,Source name,Source URL,File Date,Non-ASCII Chars
0,1,Data Engineer,"""ALL our jobs are US based and candidates must...",2023-06-11,2023-06-11,23,2023,6,2023Q2,TechFetch.com - On Demand Tech Workforce hirin...,...,NaN,NaN,IT Services,Statistician Data Science,"Bachelor, Ab Initio, Big Data, Healthcare Insu...","Databases, Computer Science, Information Techn...",LinkedIn,https://www.linkedin.com/jobs/view/data-engine...,2023-04-01,[]
1,2,Data Scientist,"""ALL our jobs are US based and candidates must...",2023-06-11,2023-06-11,23,2023,6,2023Q2,TechFetch.com - On Demand Tech Workforce hirin...,...,NaN,NaN,IT Services,Statistician AI,"Transformation, Sas, Azure, Databricks, Data L...","Support, Business Intelligence, Server, Langua...",LinkedIn,https://www.linkedin.com/jobs/view/data-scient...,2023-04-01,[]


---
# Break out TikTok entries

In [ ]:
# Filter by "Hiring entity" to include only "TikTok"
tiktok_jobs = df_sample[df_sample['Hiring entity'].str.contains("TikTok", case=False, na=False)]
print(f'TikTok jobs: {len(tiktok_jobs)}')

TikTok jobs: 1640


In [ ]:
# prompt: remove duplicates based on "Full job description"
#
tiktok_jobs = tiktok_jobs.drop_duplicates(subset=['Full job description'])
print(f'Deduped TikTok jobs: {len(tiktok_jobs)}')

Deduped TikTok jobs: 1633


In [ ]:
# Check date range
print(f"First TikTok job posting: {tiktok_jobs['Date posted'].min()}")
print(f"Last TikTok job posting: {tiktok_jobs['Date posted'].max()}")


First TikTok job posting: 2022-02-10 00:00:00
Last TikTok job posting: 2024-06-17 00:00:00


In [ ]:
# prompt: Print the number of jobs per month within tiktok_jobs using "Date posted"
#
tiktok_jobs['Date posted'] = pd.to_datetime(tiktok_jobs['Date posted'])
tiktok_jobs_per_quarter = tiktok_jobs.groupby(pd.Grouper(key='Date posted', freq='Q')).size()
print(tiktok_jobs_per_quarter)


Date posted
2022-03-31     22
2022-06-30     44
2022-09-30    158
2022-12-31    257
2023-03-31    188
2023-06-30    117
2023-09-30    262
2023-12-31    264
2024-03-31    204
2024-06-30    117
Freq: Q-DEC, dtype: int64


---
# Set up OpenAI and Pydantic


In [ ]:
# Set up a new, simplified Pydantic class
#
class Title(BaseModel):
    """
    Represents the full title of the job posting.

    This model captures the full title of the job posting
    Examples include:
    'Senior Machine Learning Engineer, TikTok Ads Core- Business Account',
    'Lead Research Engineer, Machine Learning - TikTok Privacy Innovation Lab'
    """
    title: str # Added type annotation for title

class Role(BaseModel):
    """
    Represents the area in which the employee will work.

    This model capture the area, department, or division in which the employee
    will work within the company. Examples include:
    'Research and development', 'User engagement', 'Systems infrastructure',
    'Trust and safety'
    """
    role: str # Added type annotation for role

class Objectives(BaseModel):
    """
    Represents the stated objectives the company has set for the employee.
    Examples include:
    ' 1. Drive insightful understanding of Live users and creators to accelerate
    better user, creator experience in Live.', '2. Keep track of core metrics and
    attribute metric changes to root causes.', '3. Use data and\nexperiments to
    verify hypotheses around bottlenecks in product growth, and consult on the
    direction for improvement.', '4. Work with MLE and product stakeholders to
    deLiver product insights and strategy.'
    """
    objectives: list[str] # Added type annotation for objectives

class Technologies(BaseModel):
    """
    Represents a technology-based skills required by a hiring entity.

    This model captures basic information about a products and technologies
    mentioned within this job post. Examples include:
    'Spark', 'Hadoop', 'PyTorch', 'Git', 'OpenAI', 'Vertex AI', 'Google Cloud',
    'Python', 'SQL', 'Llama', 'LangChain'
    """
    technologies: list[str]  # Added type annotation for technologies

class Skills(BaseModel):
    """
    Represents a technology-based skills required by a hiring entity.

    This model captures basic information about a business skills mentioned
    within this job post. Examples include:
    'data collection', 'data cleaning', 'data analysis', 'reporting',
    'data visualization', 'training', 'stakeholder collaboration', 'MLOps',
    'DataOps', 'Responsible AI', 'Data Engineering', 'Data Science', 'Data Warehousing'
    """
    skills: list[str]  # Added type annotation for skills

class Summary(BaseModel):
    """
    Represents the area in which the employee will work.

    This model capture the a concise, 250 word summary of the job posting, focusing
    on the job itself rather than the company or the job requirements and benefits.'
    """
    summary: str # Added type annotation for role


class Categories(BaseModel):
    """
    Represents the top three categories of the job posting.

    This model captures the most applicable categories to which this job posting belongs.
    The full list of five main categories, each with several sub categories to choose from is below.
    Please select a maximum of three main + sub categories such as 'Trust and Safety - Privacy',
    'Monetisation - Payments', etc.  Only pick a category + sub category if it figures into the
    job in a prominant manner. Return the top three categories and no more. Please only return a
    simple python list of strings.
    {category_list}
    """
    categories: list[str] # Added type annotation for categories

class Team(BaseModel):
    """
    Represents the team wherein the employee will work.

    This model capture the area, department, or division in which the employee
    will work within the company, using the following list. Don't worry about finding
    an exact match. But please assign this job to one of the teams listed below rather
    than making up a new team name. Just assign this post to the single closest match
    based on the following list.
    {team_list}
    """
    team: str # Added type annotation for teams

class isGenAI(BaseModel):
    """
    Represents the team wherein the employee will work.

    This model captures whether or not a job post concerns Generative AI (GenAI)
    both in terms of the technologies used and the use case at hand.
    """
    isgen: bool


class JobResponse(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates information about title, role, objectives, technologies,
    and skills mentioned in a job post
    """
    title: str
    role: str
    objectives: list[Objectives]
    technologies: list[Technologies]
    skills: list[Skills]
    summary: str
    categories: list[Categories]
    team: str
    isgen: bool


In [ ]:
# Inference function definition
#
client = OpenAI()

# set variable defaults
title = ""
role = ""
objectives = []
technologies = []
skills = []
summary = ""
categories = []
team = ""
isgen = True

def parse_post(jobpost):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that \
            scans job posts for organization info, the technologies and products, \
            and business skills required for a given job posting. Please be thorough \
            and include as many technologies and skills as you can in your response. \
            Do not include the examples UNLESS they actual appear in the jobpost. \
            And remember, this is an analysis, so be detailed and concise, and \
            above all else, do not hallucinate."},
            {"role": "user", "content": jobpost},
        ],
        response_format=JobResponse,
    )

    message = completion.choices[0].message
    if message.parsed:
        # Separate lists for technologies and business skills
        title = str(message.parsed.title)
        role = str(message.parsed.role)
        objectives = [str(obj) for obj in message.parsed.objectives]
        technologies = [str(tech) for tech in message.parsed.technologies]
        skills = [str(skill) for skill in message.parsed.skills]
        summary = str(message.parsed.summary)
        categories = [str(cat) for cat in message.parsed.categories]
        team = str(message.parsed.team)
        isgen = bool(message.parsed.isgen)

        # Return as separate values
        return title, role, objectives, technologies, skills, summary, categories, team, isgen
    else:
        return "", "", [], [], [], "", [], "", False  # Return empty lists in case of refusal or no parsing


In [ ]:
# prompt: Create df with five random rows from df_sample
#
df = tiktok_jobs.sample(n=5)
df.shape

(5, 31)

In [ ]:
# Apply the parse_post_simple function to each job post
#
df['title'], df['role'], df['objectives'], df['technologies'], df['skills'], df['summary'], \
df['categories'], df['team'], df['isgen'] = zip(*df['Full job description'].apply(parse_post))


In [ ]:
# Check results
#
def check_results(df, record):
    """Check the results of the parse_post_simple function."""
    print(f"title: {df.title.iloc[record]}")
    print(f"role: {df.role.iloc[record]}")
    print(f"objectives: {df.objectives.iloc[record]}")
    print(f"technologies: {df.technologies.iloc[record]}")
    print(f"skills: {df.skills.iloc[record]}")
    print(f"summary: {df.summary.iloc[record]}")
    print(f"categories: {df.categories.iloc[record]}")
    print(f"team: {df.team.iloc[record]}")
    print(f"isgen: {df.isgen.iloc[record]}")


In [ ]:
# prompt: run check_results for each row in df
#
for i in range(len(df)):
  check_results(df, i)
  print("\n")


title: Machine Learning Engineer - Search Growth E-commerce
role: Machine Learning Engineer
objectives: ['objectives=["Enhance search recommendation services and models for TikTok, driving increased search traffic on TikTok and TikTok Mall, while also improving user\'s search understanding.", \'Optimize the recommender system based on hyperscale machine learning models, covering a range of tasks from recall, first-stage ranking to final-stage ranking in the end-to-end workflow.\', \'Explore the upper limits of short text recommendation and general recommendation technology, with a focus on the interaction between recommendation and NLP technology.\', \'Provide guidance and mentorship to fellow teammates.\']']
technologies: ["technologies=['machine learning', 'natural language processing (NLP)', 'computer vision (CV)', 'recommendation systems', 'multi-modal technology']"]
skills: ["skills=['coding', 'data structure', 'algorithms', 'analysis', 'modeling', 'problem-solving', 'product sens

# Run against entire TikTok dataframe

In [ ]:
# Convert sample_jobs into df for processing
#
df = tiktok_jobs.copy()
df.shape


(1633, 31)

In [ ]:
# Apply the parse_post_simple function to each job post
#
df['title'], df['role'], df['objectives'], df['technologies'], df['skills'], df['summary'], \
df['categories'], df['team'], df['isgen'] = zip(*df['Full job description'].apply(parse_post))


In [35]:
# prompt: run check_results for each row in df
#
df_check = df.sample(n=5)
for i in range(len(df_check)):
  check_results(df_check, i)
  print("\n")


title: Project Manager - Trust & Safety
role: Project Manager
objectives: ["objectives=['Responsible for the operation of labeling projects, negotiating with content strategy teams, and driving the implementation and execution of requirements.', 'Proactively collaborate with cross-functional departments for project evaluation, resources coordination and subsequent works.', 'Deeply understand the implementation of frontline labeling teams and work with the demand side to develop and optimize labeling guidelines and SOPs.', 'Conduct quality management, progress management and risk management by data analysis.', 'Continuously optimize project processes and improve the efficiency of project implementation through analysis of business conditions.']"]
technologies: ["technologies=['Excel', 'Word']"]
skills: ["skills=['project management', 'data processing', 'data analysis', 'communication skills', 'resilience', 'adaptability']"]
summary: The Project Manager for TikTok's Trust & Safety team w

In [40]:
# prompt: print the top 20 job titles where isgen == True

# Filter the DataFrame for jobs where isgen is True
genai_jobs = df[df['isgen'] == True]

# Get the top 20 job titles
top_titles = genai_jobs['title'].value_counts().head(10)

# Print the results
print(f"Job Titles featuring GenAI: {len(genai_jobs)}")
print(top_titles)


Job Titles featuring GenAI: 24
title
University Graduate Machine Learning Engineer                        4
Data Scientist                                                       2
Data Analyst for Product Analytics Team                              1
Software Development Graduate Position                               1
Machine Learning Engineer - NLP                                      1
Machine Learning Engineer - E-commerce Supply Chain and Logistics    1
AI Customer Service Engineer                                         1
Product Manager, Monetization GenAI                                  1
Machine Learning Engineer - Generative AI                            1
Machine Learning Engineer - Generative AI Team                       1
Name: count, dtype: int64


In [41]:
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Job title category', 'AI technologies',
       'AI skills', 'Source name', 'Source URL', 'File Date',
       'Non-ASCII Chars', 'title', 'role', 'objectives', 'technologies',
       'skills', 'summary', 'categories', 'team', 'isgen'],
      dtype='object')

In [42]:
# Drop 'unwanted columns
#
df_clean = df.drop(['Job title category', 'AI technologies', 'AI skills', 'Non-ASCII Chars'], axis=1)
df_clean.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Source name', 'Source URL', 'File Date',
       'title', 'role', 'objectives', 'technologies', 'skills', 'summary',
       'categories', 'team', 'isgen'],
      dtype='object')

In [43]:
# Write our new df to disk
#
df_clean.to_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_13sep24.pkl")
df_clean.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_13sep24.csv")

---
# Summarizing and analyzing our full list of jobs

In [87]:
# prompt: create a dataframe that samples 50% of all rows using a consistent number of posts for a given day.
#
# Calculate the number of posts per day
posts_per_day = df_clean['Date posted'].value_counts()

# Calculate the number of posts to sample per day
sample_posts_per_day = (posts_per_day / 20).apply(np.ceil).astype(int)

# Create a new dataframe with sampled rows
sampled_df = df_clean.groupby('Date posted').apply(lambda x: x.sample(n=min(len(x), sample_posts_per_day[x.name])))
sampled_df = sampled_df.reset_index(drop=True)
sampled_df.shape

(432, 36)

In [88]:
sampled_df['Date posted'].value_counts().mean()

1.0261282660332542

In [89]:
# Combine our test data into a single block of text
#
job_data = "\n\n".join([
    f"Title: {row['title']}\n"
    f"Date posted: {row['Date posted']}\n"
    f"Role: {row['role']}\n"
    f"Objectives: {row['objectives']}\n"
    f"Technologies: {row['technologies']}\n"
    f"Skills: {row['skills']}\n"
    f"Summary: {row['summary']}\n"
    f"Categories: {row['categories']}\n"
    f"Team: {row['team']}\n"
    f"Is GenAI: {row['isgen']}\n"
    for index, row in sampled_df.iterrows()
])
print("all done")

all done


In [82]:
# prompt: print the first few lines of text from job_data
#
print(job_data[:5000]) # Print first 500 characters


Title: Software Engineer - Brand Ads
Date posted: 2022-02-10 00:00:00
Role: Engineer
Objectives: ["objectives=['Create innovative monetization products that drive engagement and revenue.', 'Participate in the development of a large-scale Ads system.', 'Participate in the development and iteration of Ads algorithms using Machine Learning.', 'Work on NLP and CV related technology for content understanding and taxonomy.', 'Contribute to the success of a rapidly growing and evolving organization with speediness and quality.']"]
Technologies: ["technologies=['Go', 'C', 'C++', 'Python', 'Machine Learning', 'Deep Learning', 'NLP', 'Content Understanding', 'Computer Vision']"]
Skills: ["skills=['Programming', 'Debugging', 'Optimization', 'Machine Learning', 'Data Mining', 'Data Analysis', 'System Design', 'Algorithm Design']"]
Summary: TikTok's Brand Ads team is looking for seasoned engineers to create monetization products that enhance user engagement and revenue, developing algorithms and sy

In [68]:
# Generate prompt with job data
#
system_prompt = f"""
You are an AI assistant specializing in analyzing job data to understand a company's strategic focus and workforce priorities.
Given multiple job postings with roles, objectives, technologies, skills, and summaries, your task is to generate a comprehensive markdown summary
detailing the overall nature of the job roles and what the company is focusing on. And remember, this is an analysis, so be detailed and concise,
and above all else, do not hallucinate.

Use the following outline to structure the analysis:

1. **Introduction**
Provide an introduction about TikTok’s current focus and strategic areas based on the job data analyzed.

2. **Job Roles and Objectives**
Summarize the main job roles within the data set, detailing the key objectives of these roles. Highlight common themes and areas of strategic emphasis.

3. **Technologies and Tools**
Identify the technologies and tools most frequently mentioned across the job posts. Explain how these technologies contribute to TikTok’s overall strategy.

4. **Skills in Demand**
Outline the critical skills that TikTok seeks in its employees, both technical and soft skills. Discuss how these skills align with the company’s objectives.

5. **Strategic Focus Areas**
Discuss what the job data suggests about TikTok’s strategic focus.

6. **Categories**
Discuss the most applicable categories to which this job posting belongs.

7. **Team**
Discuss the team wherein the employee will work.

8. **GenAI**
Discuss whether or not a job post concerns Generative AI (GenAI) both in terms of the technologies used and the use case at hand.

9. **Evolution of TikTok AI staffing practices**
Provide an analysis of the way TikTok's AI professional hiring practices have changed, highlighting any emerging trends or notable observations.

10. **Conclusion**
Provide a conclusion summarizing TikTok’s overarching goals based on the job data, highlighting any emerging trends or notable observations.

**Output Format**: Provide a markdown-formatted analysis based on the data.
"""


In [90]:
# Calculate the number of tokens we'll be sending to the model in our job_data prompt
#
# Choose the appropriate model's tokenizer
encoding = tiktoken.encoding_for_model("gpt-4o-mini")  # or whatever mode model

# Encode the prompt to get the list of tokens
tokens = encoding.encode(system_prompt + job_data)

# Calculate the number of tokens
num_tokens = len(tokens)

print(f"Number of tokens in the prompt: {num_tokens}")


Number of tokens in the prompt: 121107


In [91]:
# prompt: save job_data to a text file
#
with open('/content/gdrive/MyDrive/Colab Notebooks/Datasets/tik_tok_job_data_13sep24_sampled.txt', 'w') as f:
  f.write(job_data)


In [93]:
# set up inference
#
def analyze_vendor_jobs(system_prompt, job_data):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": job_data},
        ],
    )
    return completion.choices[0].message.content


In [94]:
# Run analysis
#
%%time
analysis = analyze_vendor_jobs(system_prompt, job_data)


CPU times: user 292 ms, sys: 40.8 ms, total: 332 ms
Wall time: 50.8 s


In [95]:
# Show results
#
print(analysis)

# TikTok Job Data Analysis

## 1. Introduction
Based on the analysis of TikTok's job postings, it is evident that the company is heavily invested in advancing its technological capabilities, particularly in automation, machine learning, and data analytics. The growing demand for roles that emphasize user experience, safety, and e-commerce functionalities indicates TikTok's efforts to maintain its competitive edge in the ever-evolving digital landscape.

## 2. Job Roles and Objectives
The job postings reflect a diverse spectrum of roles, including:
- **Software Engineers**: Focused on building scalable systems supporting diverse functionalities, especially around e-commerce and ads.
- **Data Scientists**: Engaged in analytics, risk management, and product insights, driving data-informed decisions.
- **Machine Learning Engineers**: Concentrated on improving models and algorithms for recommendation systems, ads delivery, and content safety.
- **Engineers in Infrastructure, Security, and O